In [293]:
### IMPORTING DATA functions

import scipy
from scipy import signal
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
import glob



# samplerate of the data
samplerate = 44100.
# bit depth of the data
bitdepth = np.int16

# input: a .wav file 
# output: a vector of integers (one float per sample)
def import16monowav(wav_file) :
    samplerate, data = wavfile.read(wav_file)
    #return np.fromfile(open(wav_file),bitdepth)[24:]
    return data

def import16stereo1wav(wav_file) :
    samplerate, data = wavfile.read(wav_file)
    #return np.fromfile(open(wav_file),bitdepth)[24:]
    res = [n for n,m in data]
    return res

# normalize to a float in [-1,1]
def normalize16bits(vector) :
    return [ sample/2**16. for sample in vector ]

# kills first nad last samples under some threshold
def killsilence_wav(wav_16bits) :
    # TODO: veeery arbitrary
    threshold = 30
    first = 0
    last = len(wav_16bits)-1
    while (abs(wav_16bits[first]) < threshold) :
        first += 1
    while (abs(wav_16bits[last]) < threshold) :
        last -= 1
    if first >= last :
        raise ValueError('Empty sample!')
    return wav_16bits[first:last]

# convert to float
# TODO: the silence should be removed later?
def convert_to_float(ex) :
    if channels == 1 :
        return normalize16bits(killsilence_wav(import16monowav(ex)))
    else :
        return normalize16bits(killsilence_wav(import16stereo1wav(ex)))
        
    
# import wav, convert to float, and fourier transform
def import_convert_transform(ex) :
    # import wav and convert to float
    ex_float = convert_to_float(ex)
    return transform_floats(ex_float)



#parameter for tukey window
alpha_tukey = 0.05

# given a signal of floats, apply window and transform
def transform_floats(ex_float) :
    # apply tukey window
    ex_windowed = signal.tukey(len(ex_float),alpha=0.05) * ex_float  
    # transform to frequency domain (since this is a real signal the transformed vector has half of the length)
    ex_transformed_complex = np.fft.rfft(ex_windowed)
    # take absolute values
    ex_transformed = map(abs,ex_transformed_complex)
    # compute frequencies for the xlabel
    freq_label = np.fft.rfftfreq(len(ex_float),1/samplerate)
    
    plt.plot(ex_float)
    plt.show()
    return ex_float, ex_transformed, freq_label
    #freq_label = [ np.arange(len(ex))/(len(ex)*2/samplerate) for ex in ex_transformed ]

In [301]:
### FINIDING HARMONICS functions


# TODO: this should not be sooo hardcoded
peak_width_min = 10
peak_width_max = 60

# a quarter of tone
sm_tone = 2**(1/12.)


# range of harmonics of interest
# TODO: hardcode this better
relevant_range_min = 100
relevant_range_max = 15000
# how many harmonics we want to take into account
num_harmonics = 7


# given a signal return a list of the positions of the peaks
def find_peaks_in_range(ex, frq_lbl) :
    # find peaks (this is the position in the array)
    peaks = signal.find_peaks_cwt(ex, np.arange(peak_width_min,peak_width_max))
    peaks_in_range = filter(lambda n : frq_lbl[n]>=relevant_range_min and frq_lbl[n]<= relevant_range_max, peaks)
    return peaks_in_range


def find_tonic(amplitudes) :
    maximum = max(amplitudes)
    arbitrary = 2.
    first = 0
    while (amplitudes[first]<maximum/arbitrary) :
        first += 1
    if first >= len(amplitudes) :
        raise ValueError('No tonic!')
    return first


# TODO: should be called harmonics_amplitudes
def harmonics_frequency_volumes(ex, frq_lbl) :
    # filter peaks in range
    peaks_position = find_peaks_in_range(ex, frq_lbl)
    
    # compute energy of peaks (root mean square):
    peaks_volume_tmp = [ get_rms_peak_log(ex,peak) for peak in peaks_position ]
    
    # find tonic
    tonic_pos = find_tonic(peaks_volume_tmp)
    #take only num_harmonics peaks
    peaks_volume = peaks_volume_tmp[tonic_pos:tonic_pos+num_harmonics]
    # normalize
    peaks_volume_nor = peaks_volume/rms(peaks_volume)
        ## put volumes in range
        ##peaks_volume_in_range = [ v-min_volume if v>min_volume else 0 for v in peaks_volume ]
    #the frequency of the peaks
    peaks_frequency = [ frq_lbl[n] for n in peaks_position[tonic_pos:tonic_pos+num_harmonics] ]
        # calculate volume of peaks relative to tonic
        #tonic_volume = peaks_volume[0]
        #peaks_volume_nor = [ harmonic_volume/tonic_volume for harmonic_volume in peaks_volume ]
    return peaks_frequency, peaks_volume_nor


# TODO: see if this parameter is ok
tolerance_har = 0.2


def cast_harmonics(peaks_frequency, peaks_volume) :
    har_f_har_v = zip(peaks_frequency, peaks_volume)
    tonic_freq = har_f_har_v[0][0]
    peaks_volume_res = np.zeros(num_harmonics)
    peaks_frequency_new = [ tonic_freq * n for n in range(1,num_harmonics+1)]
    
    for freq, vol in har_f_har_v :
        harm_num = int(round(freq/tonic_freq))
        if abs(freq/tonic_freq - harm_num) > tolerance_har or harm_num > num_harmonics :
            print "WARNING: strange harmonic: " + str(freq/tonic_freq)
            # raise ValueError('Strange harmonic!')
        else :
#            print harm_num
            peaks_volume_res[harm_num-1] = vol
            peaks_frequency_new[harm_num-1] = freq
        
    return peaks_frequency_new, peaks_volume_res
            
        

def get_average_volume(ex, peak) :
    # TODO: this 10 is pretty arbitrary
    windowinitial = int(peak-peak_width_max/2) 
    windowfinal = int(peak+peak_width_max/2)
    return np.average(ex[windowinitial:windowfinal])
 
# average over a tone centered in the note
def get_average_volume_log(ex, peak) :
    windowinitial = int(peak/sm_tone) 
    windowfinal = int(peak*sm_tone)
    return np.average(ex[windowinitial:windowfinal])

# rms over a neigborhood of the peak
def get_rms_peak(ex, peak) :
    # TODO: this 10 is pretty arbitrary
    windowinitial = int(peak-peak_width_max/2) 
    windowfinal = int(peak+peak_width_max/2)
    return rms(ex[windowinitial:windowfinal])
 
# rms over a tone centered in the note
def get_rms_peak_log(ex, peak) :
    windowinitial = int(peak/sm_tone) 
    windowfinal = int(peak*sm_tone)
    return rms(ex[windowinitial:windowfinal])
  
# with a change of parameter
# TODO: do the integral with np.average
def get_average_volume_log_log(ex, peak) :
    # v_n(x) = v_f(2^x),  thus \int v_n(x) = \int v_f(2^x) = \int v_f(y)/y
    windowinitial = int(peak/sm_tone) 
    windowfinal = int(peak*sm_tone)
    lenintegral = windowfinal - windowinitial
    weights = [ 1/float(y) for y in range(windowinitial,windowfinal) ]
    totalweights = sum(weights)
    averagevol = sum([v*w for v,w in zip(ex[windowinitial:windowfinal], weights)])/totalweights
    return averagevol


def rms(vec) :
    return np.sqrt(np.average(np.power(vec,2)))

In [291]:
### PLOTTING functions



# plot (1/4 of the) spectrum 
def plotspec() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,len(names_and_graphs_and_harm_number))))
    for (name, ex, freqs), (x,y) in names_and_graphs_and_harm_number :
        c = next(color)
        plt.plot(freqs, ex, label=name,c=c)
    
# logarithmic scale for x axis [log(frequency) = note]
def plotspec_log() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,len(names_and_graphs_and_harm_number))))
    for (name, ex, freqs), (x,y) in names_and_graphs_and_harm_number :
        c = next(color)
        plt.plot(np.log2(freqs), ex, label=name,c=c)
    

# logarithmic scale for both axes [log(frequency) = note, log(amplitude) = volume]
def plotspec_loglog() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,len(names_and_graphs_and_harm_number))))
    for (name, ex, freqs), (x,y) in names_and_graphs_and_harm_number :
        ex = np.log2(ex)
        c = next(color)
        plt.plot(np.log2(freqs), ex, label=name,c=c)
 
    
def plotspec_normalized() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,len(names_and_graphs_and_harm_number))))
    for (name, ex, freqs), (x,y) in names_and_graphs_and_harm_number :
        c = next(color)
        #normalize
        ex /= np.max(np.abs(ex),axis=0)
        plt.plot(freqs, ex, label=name,c=c)

def plotspec_and_harmonics() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,len(names_and_graphs_and_harm_number))))
    for (name, ex, freqs), (x,y) in names_and_graphs_and_harm_number :
        c = next(color)
        #normalize
        ex /= rms(ex)
        plt.plot(freqs, ex, label=name,c=c)
        plt.scatter(x,y , label=name,c=c)
#        plt.ylim([0,1])

    
def plotspec_and_harmonics_log() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,len(names_and_graphs_and_harm_number))))
    for (name, ex, freqs), (x,y) in names_and_graphs_and_harm_number :
        c = next(color)
        # in decibels
        ex = np.log2(ex)
        # normalize
        ex /= np.max(np.abs(ex),axis=0)
        plt.plot(freqs, ex, label=name,c=c)
        plt.scatter(x, y, label=name,c=c)
#        plt.ylim([0,1])
    
# no color version
def plotharmonics_nc() :
    for (name, ex, freqs), (x,y) in names_and_graphs_and_harm_number :
        xx = range(len(y))
        plt.scatter(xx, y,label=name)
    
def plotharmonics() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,len(names_and_graphs_and_harm_number))))
    for (name, ex, freqs), (x,y) in names_and_graphs_and_harm_number :
        c = next(color)
        xx = range(len(y))
        plt.scatter(xx, y,label=name,c=c)
    

def plotharmonics2() :
    color = iter(plt.cm.rainbow(np.linspace(0,1,len(names_and_graphs_and_harm_number))))
    for (name, ex, freqs), (x,y) in names_and_graphs_and_harm_number :
        c = next(color)
        plt.plot(y,label=name,c=c)

In [253]:
### IMPORTING SYNTHETIC SINUSOIDS functions and call

Ts = 1.0/samplerate

def gimme_sinusoid(freq) :
    t = np.arange(0,1,Ts)
    return np.sin(2*np.pi * freq * t)
    
def gimme_sinusoids(freq,harmonics_amp) :
    n = 2
    res = gimme_sinusoid(freq)
    for amp in harmonics_amp : 
        res += amp * gimme_sinusoid(n*freq)
        n += 1
    return res

def gimme_sinusoids_noise(freq,harmonics_amp) :
    signal = gimme_sinusoids(freq,harmonics_amp)
    return signal + np.random.normal(0,.5,len(signal))

        
algunos = [gimme_sinusoids_noise(500,[.8,.4,.8,.8,.6,.5,.1,.1]),
          gimme_sinusoids(400,[.5,.3,.4,.5,.6,.7,.1,.1])]
namos = ['500','400']

# more global variables
ex_floats, ex_transformed, freq_label = zip(*map(transform_floats, algunos))
names_and_graphs = zip(namos, ex_transformed, freq_label)

#plt.plot(ex_transformed[0])
#plt.show()


In [294]:
### IMPORTING DATA calls


# global variables
# directory with data
datadirectory = "./sonatina_symphonic_orchestra/Samples/Flute/"

# maximum of examples
max_ex = 1

# channels
channels = 2

# file names
ex_wav = glob.glob(datadirectory + '*.wav')[:max_ex]

# TODO: a human-readble version of ex_wav
# ex_names = 



# more global variables
ex_floats, ex_transformed, freq_label = zip(*map(import_convert_transform, ex_wav))
names_and_graphs = zip(ex_wav, ex_transformed, freq_label)


In [302]:
### FINDING HARMONICS calls

harmonic_volumes_every_example = [ harmonics_frequency_volumes(ex_tr, freq_lbl) for ex_name , ex_tr, freq_lbl in names_and_graphs ]
harmonic_number_volumes_every_example = [ cast_harmonics(f,v) for f,v in harmonic_volumes_every_example ]


In [303]:
### PLOTTING calls

# get data
names_and_graphs_and_harm = zip(names_and_graphs, harmonic_volumes_every_example)
names_and_graphs_and_harm_number = zip(names_and_graphs, harmonic_number_volumes_every_example)

# plot

#plotharmonics_nc()
#plotharmonics()
#plotharmonics2()
plotspec_and_harmonics()

#plt.legend(loc='upper right')
#plt.yscale('log')
#plt.xlim([20,6000])
plt.show()

In [140]:
#harmonics_volumes(ex_transformed[1],freq_label[1])
get_average_volume(ex_transformed[0],600)


0.0584473370517 [0.001718213058419244, 0.0017152658662092624, 0.0017123287671232876, 0.0017094017094017094, 0.0017064846416382253, 0.0017035775127768314, 0.0017006802721088435, 0.001697792869269949, 0.001694915254237288, 0.001692047377326565, 0.0016891891891891893, 0.0016863406408094434, 0.0016835016835016834, 0.0016806722689075631, 0.0016778523489932886, 0.0016750418760469012, 0.0016722408026755853, 0.001669449081803005, 0.0016666666666666668, 0.0016638935108153079, 0.0016611295681063123, 0.001658374792703151, 0.0016556291390728477, 0.001652892561983471, 0.0016501650165016502, 0.0016474464579901153, 0.001644736842105263, 0.0016420361247947454, 0.001639344262295082, 0.0016366612111292963, 0.0016339869281045752, 0.0016313213703099511, 0.0016286644951140066, 0.0016260162601626016, 0.0016233766233766235]
582 617


0.31333697161760382

In [295]:
ex_wav
#len(names_and_graphs[0])

['./sonatina_symphonic_orchestra/Samples/Flute/flute-f#3.wav']

In [110]:
plt.plot(names_and_graphs[0][1])
plt.show()

In [28]:
#samplerate, data = wavfile.read(ex_wav[2])


def killsilence(wav_16bits) :

    threshold = 30
    
    first = 0
    last = len(data)-1
    
    while (wav_16bits[first] < threshold) :
        first += 1
        
    while (wav_16bits[last] < threshold) :
        last -= 1

ex_float = normalize16bits(killsilence_wav(import16monowav(ex_wav[0])))
# apply tukey window
ex_windowed = signal.tukey(len(ex_float),alpha=0.05) * ex_float 
        
plt.plot(ex_windowed)
plt.show()


In [85]:
range(1,2+1)

[1, 2]

In [ ]:
#smooth vector, but the peak finder already does this

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

average_factor = 30

ex_smoothened = [ smooth(ex, average_factor) for ex in ex_transformed ]


In [ ]:
#TODOs:
# - there is a problem with the last harmonic, it is always 0